In [1]:
print("Hello World")

Hello World


In [28]:
with open('api_keys.txt', 'r') as api_key_file:
   api_key = api_key_file.read().strip()

In [1]:
%pip install simplejson

  Obtaining dependency information for simplejson from https://files.pythonhosted.org/packages/ca/c7/a530c9aba6a47823df3d46f20ce09be730d04aeb67aaef057a7c515654bd/simplejson-3.19.2-cp310-cp310-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import requests

In [42]:
endpoint = "https://api.fda.gov/food/enforcement.json"

# Set your query parameters
params = {
    'search': 'state:UT',
    'limit': 200  # Adjust the limit based on your needs
}

headers = {
    'Content-Type': 'application/json',
    'api_key': api_key
}

response = requests.get(endpoint, params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.status_code)

In [43]:
import pandas as pd

results = data.get('results', [])

df = pd.DataFrame(results)

In [44]:
print(df)

           country              city           address_1  \
0    United States            Kearns       4850 S 5300 W   
1    United States            Murray        747 W 4170 S   
2    United States     American Fork         708 S 500 E   
3    United States    Salt Lake City   440 W Lawndale Dr   
4    United States       Springville        1086 N 450 W   
..             ...               ...                 ...   
170  United States    Salt Lake City   440 W Lawndale Dr   
171  United States            Lindon         1453 W 40 S   
172  United States  West Valley City       2269 S 3270 W   
173  United States    Salt Lake City       3628 W 1820 S   
174  United States    Salt Lake City  3453 S West Temple   

                                     reason_for_recall address_2  \
0    Condies Foods is recalling Veggie Cups due to ...             
1    Firm received complaints of leaking, black liq...             
2    Products may be moldy prior to the end of the ...             
3    Pr